In [2]:
import pandas as pd
import scipy.stats as stats
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sms
import os
import inspect
import string

In [3]:
substances = pd.read_csv('./ISTART - RECOVERED DATA - reward sensitivity screeners.csv', header = 0)
substances

,RealID,gender,spsrq_1,spsrq_2,spsrq_3,spsrq_4,spsrq_5,spsrq_6,spsrq_7,spsrq_8,...,dudit_4,dudit_5,dudit_6,dudit_7,dudit_8,dudit_9,dudit_10,dudit_11,audit,dudit
0,1001,1,1,1,1,1,1,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,2,1,1,1,1,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,1,1,1,1,1,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,1,1,1,0,1,0,1,0,0,...,1,1,1,1,1,1,1,1,NaN,NaN
4,1005,1,0,1,0,1,1,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,3224,2,1,0,1,0,1,,0,1,...,1,1,1,1,1,1,1,1,NaN,NaN
92,3223,1,1,1,1,1,0,0,0,0,...,1,1,1,1,1,1,1,1,NaN,NaN
93,3222,2,1,1,1,1,1,1,0,0,...,1,1,1,1,1,1,1,1,NaN,NaN
94,3218,2,0,1,1,1,1,1,0,1,...,6,6,6,5,5,4,3,3,NaN,NaN


In [4]:
#function to input column name from Excel and output column number
def col2num(col):
    num = 0
    for c in col:
        if c in string.ascii_letters:
            num = num * 26 + (ord(c.upper()) - ord('A')) + 1
    return num - 1

#function to replace mis-coded AUDIT/DUDIT data with the correct integer values
def replace_num(foo):
    try:
        foo = int(foo)
        vals = {1:0, 3:1, 4:2, 5:3, 6:4}
        return vals[foo]
    except: return foo
    
#function to replace slightly differently scored mis-coded data from AUDIT/DUDIT last two columns
def replace_extra(foo):
    try:
        foo = int(foo)
        vals = {1:0, 3:2, 4:4}
        return vals[foo]
    except: return foo
    
#The below script defines the column ranges from Excel for each section of the AUDIT and DUDIT, and prints them for later use 
audit = (col2num('BY'),col2num('CG'))
dudit = (col2num('CS'),col2num('DA'))
audit_end = (col2num('CG'),col2num('CI'))
dudit_end = (col2num('DA'),col2num('DC'))

print(audit,'\n',dudit,'\n',audit_end,'\n',dudit_end)

(76, 84) 
 (96, 104) 
 (84, 86) 
 (104, 106)


In [5]:
#In a copy of the substances dataframe, the values are rescored correctly using the previous functions

sub_copy = substances.copy()

for n in range(76,84):
    col = sub_copy.iloc[:,n]
    new_col = list(map(replace_num, col))
    sub_copy.iloc[:,n] = new_col
    
for n in range(96,104):
    col = sub_copy.iloc[:,n]
    new_col = list(map(replace_num, col))
    sub_copy.iloc[:,n] = new_col
    
single_col = sub_copy['dudit_1']
new_single_col = list(map(replace_num, col))
sub_copy['dudit_1'] = new_single_col

for n in range(84,86):
    col = sub_copy.iloc[:,n]
    new_col = list(map(replace_extra, col))
    sub_copy.iloc[:,n] = new_col
    
for n in range(104,106):
    col = sub_copy.iloc[:,n]
    new_col = list(map(replace_extra, col))
    sub_copy.iloc[:,n] = new_col

#the dataframe is then cut to only include the relevant columns, and the ID column is changed to a name that matches other dataframes, on which it's merged
sub_copy.drop(sub_copy.columns[col2num('B'):col2num('BX')], axis=1, inplace = True)
sub_copy = sub_copy.rename(columns = {'RealID':'sub'})
sub_copy.drop(sub_copy.columns[1], axis=1, inplace = True)
sub_copy

,sub,audit_1,audit_2,audit_3,audit_4,audit_5,audit_6,audit_7,audit_8,audit_9,...,dudit_4,dudit_5,dudit_6,dudit_7,dudit_8,dudit_9,dudit_10,dudit_11,audit,dudit
0,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1004,2.0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,3224,0.0,0,2,1,0,0,1,1,2,...,0,0,0,0,0,0,0,0,NaN,NaN
92,3223,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
93,3222,0.0,,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
94,3218,1.0,2,0,0,1,0,2,0,0,...,4,4,4,3,3,2,2,2,NaN,NaN


In [6]:
#Here the AUDIT, DUDIT, AADIS, and 7U7D data are merged into one dataframe, and output to a .csv file

SUSD = pd.read_csv('./7U7D.csv').rename(columns = {'participant_id':'sub'})
AADIS = pd.read_csv('./AADIS.csv').rename(columns = {'participant_id':'sub'})

SubnMood = SUSD.merge(sub_copy, on = 'sub', how = 'right').sort_values('sub', ascending = True, ignore_index = True)
SubnMood = SubnMood.merge(AADIS, on = 'sub', how = 'left').sort_values('sub', ascending = True, ignore_index = True)
SubnMood.to_csv('Substance_and_Mood_data.csv', index = False)
pd.read_csv('./Substance_and_Mood_data.csv')

,sub,score_susd_q1,score_susd_q2,score_susd_q3,score_susd_q4,score_susd_q5,score_susd_q6,score_susd_q7,score_susd_q8,score_susd_q9,...,aadis_q4,aadis_q5,aadis_q6,aadis_q7,aadis_q8,aadis_q9,aadis_q10,aadis_q11,aadis_q12,aadis_q13
0,1001,2.0,3.0,0.0,1.0,3.0,0.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1002,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,3224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,3225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,3226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,3227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
